# IMGP - Semisupervised Learning - 1D Manifold

## Preamble

This notebook provides an example of how to perform Gaussian Process Regression on a 1D manifold. In this example we consider a supervised learning scenario, namely the number of labeled data points is equivalent to the number of the sampled points from the underlying manifold.

In [1]:
import math
import torch
import gpytorch
import numpy as np

from manifold_gp.kernels.riemann_matern_kernel import RiemannMaternKernel
from manifold_gp.models import RiemannGP, VanillaGP
from manifold_gp.utils import rmnist_dataset, vanilla_train, manifold_informed_train, test_model, NearestNeighbors

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Dataset

In [2]:
num_train = 0.1
scaling, single_digit, regenerate = True, True, False, 
normalize_x, normalize_y = False, True
graphbandwidth_constraint, graphbandwidth_prior = False, False
load_manifold_model, load_vanilla_model = False, False
train_manifold_model, train_vanilla_model = True, True
save_manifold_model, save_vanilla_model = True, True

In [3]:
# sampled_x, sampled_y, _, test_x, test_y, _ = rmnist_dataset(scaling=scaling, single_digit=single_digit, regenerate=regenerate)
# torch.manual_seed(1337)
# train_idx = torch.zeros(sampled_x.shape[0]).scatter_(0, torch.randperm(sampled_x.shape[0])[:int(num_train*sampled_x.shape[0])], 1).bool()
sampled_x, sampled_y = torch.from_numpy(np.load('../datasets/srmnist_train_x.npy')).float(), torch.from_numpy(np.load('../datasets/srmnist_train_y.npy')).float()
test_x, test_y = torch.from_numpy(np.load('../datasets/srmnist_test_x.npy')).float(), torch.from_numpy(np.load('../datasets/srmnist_test_y.npy')).float()
torch.manual_seed(1337)
rand_idx = torch.randperm(sampled_x.shape[0])
sampled_x, sampled_y = sampled_x[rand_idx], sampled_y[rand_idx]
train_idx = torch.zeros(sampled_x.shape[0]).scatter_(0, torch.arange(0, int(num_train*sampled_x.shape[0])), 1).bool()

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
sampled_x, sampled_y = sampled_x.contiguous().to(device).flatten(start_dim=1), sampled_y.contiguous().to(device)
train_x, train_y = sampled_x[train_idx], sampled_y[train_idx]
test_x, test_y = test_x.contiguous().to(device).flatten(start_dim=1), test_y.contiguous().to(device)
    
if normalize_x:
    mu_x, std_x = sampled_x.mean(dim=-2, keepdim=True), sampled_x.std(dim=-2, keepdim=True) + 1e-6
    sampled_x.sub_(mu_x).div_(std_x)
    train_x.sub_(mu_x).div_(std_x)
    test_x.sub_(mu_x).div_(std_x)
if normalize_y:
    mu_y, std_y = train_y.mean(), train_y.std()
    sampled_y.sub_(mu_y).div_(std_y)
    train_y.sub_(mu_y).div_(std_y)
    test_y.sub_(mu_y).div_(std_y)

In [4]:
if graphbandwidth_constraint or graphbandwidth_prior:
    knn = NearestNeighbors(sampled_x, nlist=1)
    edge_values = knn.search(sampled_x, 10)[0][:, 1:]
    
    graphbandwidth_min = edge_values[:,0].max().div(-4*math.log(1e-4)).sqrt()
    median = edge_values.sqrt().mean(dim=1).sort()[0][int(round(edge_values.shape[0]*0.50))]
    gamma_rate = 4*median/(median-graphbandwidth_min)**2
    gamma_concentration = gamma_rate * median + 1
    
    del knn, edge_values

## Model

In [5]:
%%capture
model_vanilla = VanillaGP(
    train_x, 
    train_y, 
    gpytorch.likelihoods.GaussianLikelihood(), 
    gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
    # gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(nu=2.5))
).to(device)

hypers_vanilla = {
    'likelihood.noise_covar.noise': 1e-2,
    'covar_module.base_kernel.lengthscale': 1.0,
    'covar_module.outputscale': 1.0,
}
model_vanilla.initialize(**hypers_vanilla)

In [6]:
%%capture
likelihood = gpytorch.likelihoods.GaussianLikelihood(
    noise_constraint=gpytorch.constraints.GreaterThan(1e-8),
)

kernel = gpytorch.kernels.ScaleKernel(
    RiemannMaternKernel(
        nu=2,
        x=sampled_x,
        nearest_neighbors=50,
        laplacian_normalization="randomwalk",
        num_modes=100,
        bump_scale=10.0,
        bump_decay=0.01,
        graphbandwidth_constraint=gpytorch.constraints.GreaterThan(graphbandwidth_min) if graphbandwidth_constraint else None,
        graphbandwidth_prior=gpytorch.priors.GammaPrior(gamma_concentration, gamma_rate) if graphbandwidth_prior else None
    )
)

model = RiemannGP(train_x, train_y, likelihood, kernel, train_idx).to(device)

hypers = {
    'likelihood.noise_covar.noise': 1e-2,
    'covar_module.base_kernel.graphbandwidth': kernel.base_kernel.graphbandwidth_prior.sample() if graphbandwidth_prior else 0.5,
    'covar_module.base_kernel.lengthscale': 1.0,
    'covar_module.outputscale': 1.0,
}
model.initialize(**hypers)

## Train

In [7]:
if load_manifold_model:
    model.load_state_dict(torch.load('../models/srmnist_manifold_semisupervised.pth' if single_digit else '../models/rmnist_manifold_semisupervised.pth'))

if train_manifold_model:
    optimizer = torch.optim.Adam(model.parameters(), lr=5e-2, weight_decay=0.0)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=50, threshold=1e-3, threshold_mode='rel',
                                                           cooldown=0, min_lr=0, eps=1e-8, verbose=True)
    loss = manifold_informed_train(model, optimizer, max_iter=100, tolerance=1e-2, update_norm=None, num_rand_vec=100,
                                   max_cholesky=1000, cg_tolerance=1e-2, cg_max_iter=1000, scheduler=scheduler, verbose=True)
    if save_manifold_model:
        torch.save(model.state_dict(), '../models/srmnist_manifold_semisupervised.pth' if single_digit else '../models/rmnist_manifold_semisupervised.pth')

/home/bernardo/.local/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:46: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Iteration: 0, Loss: 1823.060, Lr: 0.05,	Noise Variance: 0.010,	Signal Variance: 1.944,	Lengthscale: 1.000, Graphbandwidth: 0.500
Iteration: 1, Loss: 1653.545, Lr: 0.05,	Noise Variance: 0.011,	Signal Variance: 1.901,	Lengthscale: 1.032, Graphbandwidth: 0.520
Iteration: 2, Loss: 1530.147, Lr: 0.05,	Noise Variance: 0.011,	Signal Variance: 1.860,	Lengthscale: 1.064, Graphbandwidth: 0.540
Iteration: 3, Loss: 1446.220, Lr: 0.05,	Noise Variance: 0.012,	Signal Variance: 1.820,	Lengthscale: 1.095, Graphbandwidth: 0.560
Iteration: 4, Loss: 1387.778, Lr: 0.05,	Noise Variance: 0.012,	Signal Variance: 1.781,	Lengthscale: 1.126, Graphbandwidth: 0.579
Iteration: 5, Loss: 1355.738, Lr: 0.05,	Noise Variance: 0.012,	Signal Variance: 1.746,	Lengthscale: 1.156, Graphbandwidth: 0.597
Iteration: 6, Loss: 1337.880, Lr: 0.05,	Noise Variance: 0.013,	Signal Variance: 1.713,	Lengthscale: 1.184, Graphbandwidth: 0.613
Iteration: 7, Loss: 1331.481, Lr: 0.05,	Noise Variance: 0.013,	Signal Variance: 1.683,	Lengthscal

/home/bernardo/.local/lib/python3.10/site-packages/linear_operator/utils/linear_cg.py:338: NumericalWarning: CG terminated in 1000 iterations with average residual norm 0.188089057803154 which is larger than the tolerance of 0.01 specified by linear_operator.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a linear_operator.settings.max_cg_iterations(value) context.
  warnings.warn(


Iteration: 71, Loss: -416.178, Lr: 0.05,	Noise Variance: 0.008,	Signal Variance: 0.803,	Lengthscale: 2.811, Graphbandwidth: 0.154


/home/bernardo/.local/lib/python3.10/site-packages/linear_operator/utils/linear_cg.py:338: NumericalWarning: CG terminated in 1000 iterations with average residual norm 2.5282273292541504 which is larger than the tolerance of 0.01 specified by linear_operator.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a linear_operator.settings.max_cg_iterations(value) context.
  warnings.warn(


Iteration: 72, Loss: -532.160, Lr: 0.05,	Noise Variance: 0.009,	Signal Variance: 0.783,	Lengthscale: 2.829, Graphbandwidth: 0.149


KeyboardInterrupt: 

In [10]:
if load_vanilla_model:
    model_vanilla.load_state_dict(torch.load('../models/srmnist_vanilla_semisupervised.pth' if single_digit else '../models/rmnist_vanilla_semisupervised.pth'))

if train_vanilla_model:
    optimizer_vanilla = torch.optim.Adam(model_vanilla.parameters(), lr=1e-1, weight_decay=0.0)
    scheduler_vanilla = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_vanilla, mode='min', factor=0.5, patience=200, threshold=1e-3, 
                                                                   threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-8, verbose=False)
    loss = vanilla_train(model_vanilla, optimizer_vanilla, max_iter=100, max_cholesky=1000, tolerance=1e-2, cg_tolerance=1e-2, cg_max_iter=1000, scheduler=None, 
                  verbose=True)
    if save_vanilla_model:
        torch.save(model_vanilla.state_dict(), '../models/srmnist_vanilla_semisupervised.pth' if single_digit else '../models/rmnist_vanilla_semisupervised.pth')

Iteration: 0, Loss: -0.067, Lr: 0.1,	Noise Variance: 0.009,	Signal Variance: 0.938,	Lengthscale: 1.064
Iteration: 1, Loss: -0.140, Lr: 0.1,	Noise Variance: 0.008,	Signal Variance: 0.878,	Lengthscale: 1.131
Iteration: 2, Loss: -0.241, Lr: 0.1,	Noise Variance: 0.007,	Signal Variance: 0.821,	Lengthscale: 1.200
Iteration: 3, Loss: -0.301, Lr: 0.1,	Noise Variance: 0.007,	Signal Variance: 0.767,	Lengthscale: 1.270
Iteration: 4, Loss: -0.355, Lr: 0.1,	Noise Variance: 0.006,	Signal Variance: 0.716,	Lengthscale: 1.342
Iteration: 5, Loss: -0.415, Lr: 0.1,	Noise Variance: 0.006,	Signal Variance: 0.667,	Lengthscale: 1.416
Iteration: 6, Loss: -0.480, Lr: 0.1,	Noise Variance: 0.005,	Signal Variance: 0.620,	Lengthscale: 1.492
Iteration: 7, Loss: -0.541, Lr: 0.1,	Noise Variance: 0.005,	Signal Variance: 0.577,	Lengthscale: 1.568
Iteration: 8, Loss: -0.612, Lr: 0.1,	Noise Variance: 0.004,	Signal Variance: 0.537,	Lengthscale: 1.646
Iteration: 9, Loss: -0.685, Lr: 0.1,	Noise Variance: 0.004,	Signal Varian

/home/bernardo/.local/lib/python3.10/site-packages/linear_operator/utils/linear_cg.py:338: NumericalWarning: CG terminated in 1000 iterations with average residual norm 0.013912507332861423 which is larger than the tolerance of 0.01 specified by linear_operator.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a linear_operator.settings.max_cg_iterations(value) context.
  warnings.warn(
/home/bernardo/.local/lib/python3.10/site-packages/linear_operator/utils/linear_cg.py:338: NumericalWarning: CG terminated in 1000 iterations with average residual norm 0.014273851178586483 which is larger than the tolerance of 0.01 specified by linear_operator.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a linear_operator.settings.max_cg_iterations(value) context.
  warnings.warn(


Iteration: 91, Loss: -1.878, Lr: 0.1,	Noise Variance: 0.000,	Signal Variance: 0.290,	Lengthscale: 6.027
Iteration: 92, Loss: -1.892, Lr: 0.1,	Noise Variance: 0.000,	Signal Variance: 0.293,	Lengthscale: 6.050


/home/bernardo/.local/lib/python3.10/site-packages/linear_operator/utils/linear_cg.py:338: NumericalWarning: CG terminated in 1000 iterations with average residual norm 0.01394312921911478 which is larger than the tolerance of 0.01 specified by linear_operator.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a linear_operator.settings.max_cg_iterations(value) context.
  warnings.warn(
/home/bernardo/.local/lib/python3.10/site-packages/linear_operator/utils/linear_cg.py:338: NumericalWarning: CG terminated in 1000 iterations with average residual norm 0.011672121472656727 which is larger than the tolerance of 0.01 specified by linear_operator.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a linear_operator.settings.max_cg_iterations(value) context.
  warnings.warn(


Iteration: 93, Loss: -1.875, Lr: 0.1,	Noise Variance: 0.000,	Signal Variance: 0.296,	Lengthscale: 6.073
Iteration: 94, Loss: -1.869, Lr: 0.1,	Noise Variance: 0.000,	Signal Variance: 0.298,	Lengthscale: 6.097
Iteration: 95, Loss: -1.847, Lr: 0.1,	Noise Variance: 0.000,	Signal Variance: 0.300,	Lengthscale: 6.121
Iteration: 96, Loss: -1.868, Lr: 0.1,	Noise Variance: 0.000,	Signal Variance: 0.302,	Lengthscale: 6.146


/home/bernardo/.local/lib/python3.10/site-packages/linear_operator/utils/linear_cg.py:338: NumericalWarning: CG terminated in 1000 iterations with average residual norm 0.011595646850764751 which is larger than the tolerance of 0.01 specified by linear_operator.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a linear_operator.settings.max_cg_iterations(value) context.
  warnings.warn(
/home/bernardo/.local/lib/python3.10/site-packages/linear_operator/utils/linear_cg.py:338: NumericalWarning: CG terminated in 1000 iterations with average residual norm 0.012011639773845673 which is larger than the tolerance of 0.01 specified by linear_operator.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a linear_operator.settings.max_cg_iterations(value) context.
  warnings.warn(


Iteration: 97, Loss: -1.845, Lr: 0.1,	Noise Variance: 0.000,	Signal Variance: 0.303,	Lengthscale: 6.170
Iteration: 98, Loss: -1.870, Lr: 0.1,	Noise Variance: 0.000,	Signal Variance: 0.305,	Lengthscale: 6.194


/home/bernardo/.local/lib/python3.10/site-packages/linear_operator/utils/linear_cg.py:338: NumericalWarning: CG terminated in 1000 iterations with average residual norm 0.014803348109126091 which is larger than the tolerance of 0.01 specified by linear_operator.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a linear_operator.settings.max_cg_iterations(value) context.
  warnings.warn(
/home/bernardo/.local/lib/python3.10/site-packages/linear_operator/utils/linear_cg.py:338: NumericalWarning: CG terminated in 1000 iterations with average residual norm 0.018397025763988495 which is larger than the tolerance of 0.01 specified by linear_operator.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a linear_operator.settings.max_cg_iterations(value) context.
  warnings.warn(


Iteration: 99, Loss: -1.826, Lr: 0.1,	Noise Variance: 0.000,	Signal Variance: 0.306,	Lengthscale: 6.219
Iteration: 100, Loss: -1.866, Lr: 0.1,	Noise Variance: 0.000,	Signal Variance: 0.308,	Lengthscale: 6.245


/home/bernardo/.local/lib/python3.10/site-packages/linear_operator/utils/linear_cg.py:338: NumericalWarning: CG terminated in 1000 iterations with average residual norm 0.0153274517506361 which is larger than the tolerance of 0.01 specified by linear_operator.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a linear_operator.settings.max_cg_iterations(value) context.
  warnings.warn(


## Evaluation

In [11]:
rmse_vanilla, nll_vanilla = test_model(model_vanilla, test_x, test_y, noisy_test=True, base_model=None, max_cholesky=1000, cg_tolerance=1e-2, cg_iterations=1000)
print("RMSE Vanilla: ", rmse_vanilla)
print("NLL Vanilla: ", nll_vanilla)

RMSE Vanilla:  tensor(0.0132, device='cuda:0')
NLL Vanilla:  tensor(-2.9825, device='cuda:0')


In [12]:
rmse, nll = test_model(model, test_x, test_y, noisy_test=True, base_model=model_vanilla, max_cholesky=1000, cg_tolerance=1e-2, cg_iterations=2000)
print("RMSE Geometric: ", rmse)
print("NLL Geometric: ", nll)

RMSE Geometric:  tensor(0.1024, device='cuda:0')
NLL Geometric:  tensor(-0.8141, device='cuda:0')
